In [ ]:


from dataclasses import dataclass, field
from typing import List, Dict


# -----------------------------
# 1. Core Configuration
# -----------------------------

RACE_DISTANCE: float = 200.0   # meters
DT: float = 0.1                # simulation time step in seconds
MAX_RACE_TIME: float = 40.0    # safety cap, in seconds


# -----------------------------
# 2. Data Model
# -----------------------------

@dataclass
class BodySystemRunner:

    name: str
    system: str
    lane: int

    # Base attributes (tuned to feel like a 200m sprint, not literal real-world values)
    acceleration: float        # how quickly the runner approaches target speed
    top_speed: float           # maximum sustainable speed (m/s)
    stamina_max: float         # maximum stamina pool
    stamina_regen: float       # how fast stamina regenerates when not going all-out
    burst_multiplier: float    # how strong late/early bursts are
    fatigue_factor: float      # how strongly low stamina penalizes speed

    personality: str           # "calm", "aggressive", "tactical", "steady", etc.
    ability: str               # ability keyword, e.g. "heart_engine"

    # Dynamic race state
    position: float = 0.0      # current distance along the track (0 -> RACE_DISTANCE)
    speed: float = 0.0         # current speed (m/s)
    stamina: float = field(default=None)  # current stamina; initialized to stamina_max
    finished: bool = False
    finish_time: float | None = None

    def __post_init__(self) -> None:
        # Initialize stamina to full pool
        if self.stamina is None:
            self.stamina = self.stamina_max

    def update(self, dt: float, global_time: float, race_progress: float) -> None:
        """
        Advance this runner's state by one time step `dt`.

        :param dt: time step size in seconds
        :param global_time: time since race start in seconds
        :param race_progress: 0.0 -> 1.0 (approx) average race completion
        """
        if self.finished:
            return

        # -------------------------
        # 1. Decide target speed based on personality
        # -------------------------

        # Base "cruising" speed as a fraction of top_speed
        base_target = 0.8 * self.top_speed

        if self.personality == "aggressive":
            target_speed = min(self.top_speed, base_target + 0.15 * self.top_speed)
        elif self.personality == "calm":
            target_speed = base_target
        elif self.personality == "tactical":
            # Tactical runners conserve early, push mid/late race
            if race_progress < 0.4:
                target_speed = 0.7 * self.top_speed
            elif race_progress < 0.8:
                target_speed = 0.9 * self.top_speed
            else:
                target_speed = self.top_speed
        elif self.personality == "steady":
            target_speed = 0.85 * self.top_speed
        else:
            target_speed = base_target

        # -------------------------
        # 2. Apply special abilities (medical identity)
        # -------------------------

        # Cardiovascular - Cardio: "heart_engine" -> strong stamina regen when not max speed
        if self.ability == "heart_engine":
            if self.speed < self.top_speed:
                self.stamina += self.stamina_regen * 1.2 * dt

        # Respiratory - Lungster: "deep_inhale" -> periodic stamina spikes every ~20m
        if self.ability == "deep_inhale":
            if int(self.position) % 20 == 0 and self.position > 0:
                self.stamina += self.stamina_regen * 3.0 * dt

        # Muscular - Flexor: "power_burst" -> very strong early phase
        if self.ability == "power_burst":
            if race_progress < 0.25:
                target_speed = min(self.top_speed * 1.1, target_speed * 1.15)

        # Nervous - Neuron: "reflex_start" -> very fast initial acceleration
        if self.ability == "reflex_start":
            if global_time < 1.5:
                target_speed = min(self.top_speed, target_speed * 1.2)

        # Endocrine - Hormona: "adrenaline_surge" -> late-race burst
        if self.ability == "adrenaline_surge":
            if race_progress > 0.6:
                target_speed = min(self.top_speed * self.burst_multiplier,
                                   target_speed * self.burst_multiplier)

        # Immune - Defenda: "fatigue_shield" -> handled in fatigue section

        # Digestive - Gastro: "energy_conversion" -> when stamina low, speed buff
        if self.ability == "energy_conversion":
            if self.stamina < 0.3 * self.stamina_max:
                target_speed = min(self.top_speed * 1.05, target_speed * 1.1)

        # Skeletal - Bonestride: "structural_stability" -> handled in fatigue section

        # -------------------------
        # 3. Stamina mechanics
        # -------------------------

        # Drain depends on how close we are to top speed
        speed_fraction = self.speed / self.top_speed if self.top_speed > 0 else 0.0
        drain_rate = 0.5 + 1.5 * speed_fraction  # between ~0.5 and 2.0

        # Drain for running
        self.stamina -= drain_rate * dt

        # Natural regen if not going all-out
        if speed_fraction < 0.7:
            self.stamina += self.stamina_regen * dt

        # Clamp stamina
        if self.stamina < 0.0:
            self.stamina = 0.0
        if self.stamina > self.stamina_max:
            self.stamina = self.stamina_max

        # -------------------------
        # 4. Apply fatigue effects to target speed
        # -------------------------

        stamina_fraction = self.stamina / self.stamina_max if self.stamina_max > 0 else 0.0
        fatigue_penalty = (1.0 - stamina_fraction) * self.fatigue_factor

        # Immune - Defenda: reduced fatigue penalty
        if self.ability == "fatigue_shield":
            fatigue_penalty *= 0.4

        effective_target_speed = max(0.0, target_speed * (1.0 - fatigue_penalty))

        # Skeletal - Bonestride: structural stability, can't drop below ~90% of cruise
        if self.ability == "structural_stability":
            min_speed = 0.9 * 0.85 * self.top_speed
            if effective_target_speed < min_speed:
                effective_target_speed = min_speed

        # -------------------------
        # 5. Accelerate or decelerate toward target
        # -------------------------

        if self.speed < effective_target_speed:
            self.speed += self.acceleration * dt
            if self.speed > effective_target_speed:
                self.speed = effective_target_speed
        else:
            # Decelerate slightly faster than accelerate
            self.speed -= self.acceleration * 1.3 * dt
            if self.speed < effective_target_speed:
                self.speed = effective_target_speed

        # Clamp speed to non-negative
        if self.speed < 0.0:
            self.speed = 0.0

        # -------------------------
        # 6. Update position and finish state
        # -------------------------

        self.position += self.speed * dt

        if self.position >= RACE_DISTANCE:
            self.position = RACE_DISTANCE
            self.finished = True
            self.finish_time = global_time


# -----------------------------
# 3. Roster Setup
# -----------------------------

def create_runners() -> List[BodySystemRunner]:
    """
    Instantiate the 8 body-system runners with tuned stats.
    Lanes are 1–8.
    """
    return [
        BodySystemRunner(
            name="Cardio",
            system="Cardiovascular",
            lane=1,
            acceleration=3.0,
            top_speed=9.5,
            stamina_max=120.0,
            stamina_regen=12.0,
            burst_multiplier=1.1,
            fatigue_factor=0.5,
            personality="calm",
            ability="heart_engine",
        ),
        BodySystemRunner(
            name="Lungster",
            system="Respiratory",
            lane=2,
            acceleration=4.5,
            top_speed=9.4,
            stamina_max=80.0,
            stamina_regen=7.0,
            burst_multiplier=1.2,
            fatigue_factor=0.9,
            personality="aggressive",
            ability="deep_inhale",
        ),
        BodySystemRunner(
            name="Flexor",
            system="Muscular",
            lane=3,
            acceleration=4.2,
            top_speed=9.6,
            stamina_max=75.0,
            stamina_regen=6.0,
            burst_multiplier=1.3,
            fatigue_factor=1.0,
            personality="aggressive",
            ability="power_burst",
        ),
        BodySystemRunner(
            name="Neuron",
            system="Nervous",
            lane=4,
            acceleration=3.8,
            top_speed=9.2,
            stamina_max=90.0,
            stamina_regen=8.0,
            burst_multiplier=1.15,
            fatigue_factor=0.7,
            personality="tactical",
            ability="reflex_start",
        ),
        BodySystemRunner(
            name="Hormona",
            system="Endocrine",
            lane=5,
            acceleration=3.5,
            top_speed=9.7,
            stamina_max=85.0,
            stamina_regen=7.0,
            burst_multiplier=1.3,
            fatigue_factor=0.8,
            personality="tactical",
            ability="adrenaline_surge",
        ),
        BodySystemRunner(
            name="Defenda",
            system="Immune",
            lane=6,
            acceleration=2.8,
            top_speed=9.0,
            stamina_max=130.0,
            stamina_regen=11.0,
            burst_multiplier=1.05,
            fatigue_factor=0.3,
            personality="steady",
            ability="fatigue_shield",
        ),
        BodySystemRunner(
            name="Gastro",
            system="Digestive",
            lane=7,
            acceleration=3.6,
            top_speed=9.3,
            stamina_max=95.0,
            stamina_regen=8.5,
            burst_multiplier=1.2,
            fatigue_factor=0.8,
            personality="tactical",
            ability="energy_conversion",
        ),
        BodySystemRunner(
            name="Bonestride",
            system="Skeletal",
            lane=8,
            acceleration=2.4,
            top_speed=9.1,
            stamina_max=110.0,
            stamina_regen=10.0,
            burst_multiplier=1.1,
            fatigue_factor=0.6,
            personality="steady",
            ability="structural_stability",
        ),
    ]


# -----------------------------
# 4. Simulation Engine
# -----------------------------

def simulate_race() -> Dict[str, object]:
    """
    Run a full 200m race simulation and return:
      - frames: list of snapshots (time, positions, speeds, stamina)
      - runners: final runner states (with finish_time populated)
    This is pure backend logic, no visualization.
    """
    runners = create_runners()
    t: float = 0.0
    frames: List[Dict[str, object]] = []

    while t < MAX_RACE_TIME and not all(r.finished for r in runners):
        # race_progress ~ average completion ratio across all runners
        avg_position = sum(r.position for r in runners) / len(runners)
        race_progress = avg_position / RACE_DISTANCE if RACE_DISTANCE > 0 else 0.0

        for r in runners:
            r.update(dt=DT, global_time=t, race_progress=race_progress)

        frame = {
            "time": t,
            "positions": [r.position for r in runners],
            "speeds": [r.speed for r in runners],
            "stamina": [r.stamina for r in runners],
        }
        frames.append(frame)

        t += DT

    return {"frames": frames, "runners": runners}


# -----------------------------
# 5. Quick Test Helper
# -----------------------------

def print_race_results(sim_result: Dict[str, object]) -> None:
    """
    Convenience helper to pretty-print final standings in the notebook.
    """
    runners: List[BodySystemRunner] = sim_result["runners"]  # type: ignore

    # Sort by finish_time (DNF goes last)
    def sort_key(r: BodySystemRunner) -> float:
        return r.finish_time if r.finish_time is not None else 1e9

    ordered = sorted(runners, key=sort_key)

    print("Final Results (Backend Simulation Only)")
    print("-" * 40)
    for idx, r in enumerate(ordered, start=1):
        if r.finish_time is None:
            time_str = "DNF"
        else:
            time_str = f"{r.finish_time:.2f} s"
        print(f"{idx:>2}. {r.name:<10} [{r.system:<13}]  Time: {time_str}")



In [ ]:

if __name__ == "__main__":
    result = simulate_race()
    print_race_results(result)